In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup as bs
from urllib import request
import pandas as pd
import numpy as np

# Selenium 을 이용한 이미지 크롤링
from selenium import webdriver
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import urllib.request
import os

In [ ]:
# 검색 키워드 리스트
model_1 = [
       'XM3 E-TECH 하이브리드', 'XM3', 'QM6', 'SM6', '르노 MASTER'
        ]

In [ ]:
# 함수로 만들어서 나머지 모델 이미지 크롤링
def model_img_crawling(model):
# 1. 이미지 저장할 상위 폴더 생성
    if not os.path.isdir(f"{model}/"):
       os.makedirs(f"{model}/")

# 2. 크롬 웹드라이버 연결
    driver = webdriver.Chrome('C:/Users/Gunhee/Downloads/chromedriver_win32/chromedriver.exe')
    driver.get("https://www.google.co.kr/imghp?hl=ko")


# 3. 검색어 입력하기
    search = model
    elem = driver.find_element(By.NAME, "q")
    elem.send_keys(search)
    elem.send_keys(Keys.RETURN)


# 4. 스크롤 끝까지 내리기
    SCROLL_PAUSE_TIME = 1

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            try:
                driver.find_element(By.CSS_SELECTOR, ".mye4qd").click()
            except:
                break
        last_height = new_height



# 5. 이미지 찾아서 원본 파일로 저장하기
    images = driver.find_elements(By.CSS_SELECTOR, ".rg_i.Q4LuWd")
    count = 0

    for image in images:
        try:
            image.click()
            time.sleep(2)
            image = image.get_attribute('src')
            urllib.request.urlretrieve(image, f"{model}/" + search + "_" + str(count) + ".jpg")
            print(f"Image saved: {model}_{count}.jpg")
            count += 1
            if count > 200:
#                 raise Exception(f'{count}개 이미지를 다운로드 하였습니다.')
                break
        except:
            pass

    driver.close()


In [ ]:
# 크롤링 함수 실행
for i in model_1:
    model_img_crawling(i)

In [ ]:
# 크롤링 이미지를 넘파이

In [ ]:
import pandas as pd
import numpy as np
from urllib import request
from bs4 import BeautifulSoup

from urllib.request import Request, urlopen

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys

import time

import urllib
import cv2

from tensorflow.keras.preprocessing import image

import matplotlib.pyplot as plt

import ssl

In [ ]:
# 이미지 url을 np.array로 바꿔주는 함수
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):  
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    gcontext = ssl.SSLContext()
    webpage = urlopen(req,context=gcontext,timeout=10)  # 정상적으로 열리지 않는 url이 있어서 timeout 옵션 부여
    image = np.asarray(bytearray(webpage.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    return image

In [ ]:
car = {}
carname =['GV80','G90','G70','K5']

In [ ]:
def crawling(carname):  # 크롤링하여 dictionary 형태로 저장하는 함수
    car[carname] = []
    
    while(True):
        try:
            driver = webdriver.Chrome('chromedriver')  # 1. 구글 이미지 접속
            url = 'https://www.google.com/imghp'
            driver.get(url)
            break
        except:
            pass
        
    browser = driver.find_element(By.NAME,"q")  # 2. carname을 매개변수로 받아서 검색
    browser.send_keys(carname)
    browser.send_keys('\n')
    driver.implicitly_wait(3)
    
    selenium_scroll(driver)  # 3. 스크롤 끝까지 내리기
       
    time.sleep(3)
    
    html = driver.page_source  # 4. 스크롤 끝까지 내린 상태에서 페이지 소스 변수로 저장
    soup = BeautifulSoup(html, 'html.parser')
    
    imgs = driver.find_elements(By.CSS_SELECTOR, '#islrg > div.islrc > div a.wXeWr.islib.nfEiy')
       
    for i in range(len(imgs)):
        if i >= 500:  # 최대 500장의 이미지 저장
            break
            
        try:
            imgs[i].click()  # 5. 이미지 클릭
            time.sleep(1)
        except:
            break
        
        count = 0
        while(True):
            if count > 10:
                break
            count += 1
            try:
                # 6. 클릭한 이미지에서 url 정보 가져오기
                image = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div > div > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id > div > a')
                time.sleep(0.5)
                break
            except:
                pass
        
        count2 = 0
        while(True):
            if count2 > 5:
                break
            count2 += 1
            try:
                # 7. 가져온 url을 통해 이미지를 np.array로 저장 후 dictionary 안의 리스트에 저장
                image = image.find_element(By.TAG_NAME, 'img').get_attribute('src')
                image = url_to_image(image)
                car[carname].append(image)
                break
            except:
                pass

    driver.close()

In [ ]:
crawling(carname[-1])

In [ ]:
np.save('car project/car.npy', car)  # dictionary 저장

In [ ]:
car = np.load(('car project/car.npy'),allow_pickle='TRUE').item()  # 저장한 .npy 파일 불러오기

In [ ]:
car.keys()  # 정상적으로 불러왔는지 확인